In [1]:
import os
import tweepy

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

consumer_key= config.get('KEY','consumer_key')
consumer_secret= config.get('KEY','consumer_secret')
access_token= config.get('KEY','access_token')
access_token_secret= config.get('KEY','access_token_secret')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.get('KEY','json_loc')

auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
import datetime

x = datetime.datetime.now()
x -= datetime.timedelta(days=1)
date = x.strftime("%Y-%m-%d")
date

'2021-05-05'

In [4]:
search_words = "Delhi Health Bulletin (from:CMODelhi)"
date_since = date

tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1)

image_url = ""
# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)
    media = tweet.entities.get('media', [])
    image_url = media[0]['media_url']
    print("\n\n")

🏥Delhi Health Bulletin - 6th May 2021🏥 

#DelhiFightsCorona https://t.co/6IIGu9MtRa





In [5]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_url

response = client.text_detection(image=image)

string = ""
for text in response.text_annotations:
    string =  text.description
    print(string)
    break

GOVT. OF NCT OF DELHI
Delhi State Health Bulletin_COVID -19
(No. 428/ May 6th 2021)
COVID-19 Positive Cases Status of last 24 hours
Positive Cases
19133
Tests Conducted
78780
Positivity Rate
Recovered/Discharged/Migrated
24.29 %
20028
Deaths
335
COVID-19 Patient Management
Total beds
Оссирied
20117
Vacant
Hospital
21839
1722
Dedicated COVID Care Centre
5525
701
4824
Dedicated COVID Health Centre
206
116
90
Home Isolation
50562
COVID-19 Testing Status
RTPCR/CBNAAT/True Nat tests conducted in last 24 hours
64529
Rapid antigen test conducted in last 24 hours
14251
Total tests done so far
17597532
Tests per million
926185
COVID-19 Vaccination Status
Beneficiaries vaccinated in last 24 hours
68205
Beneficiaries vaccinated 1st dose in last 24 hours
56857
Beneficiaries vaccinated 2 nd dose in last 24 hours
11348
Cumulative beneficiaries vaccinated so far
3552037
Cumulative beneficiaries vaccinated 1st dose so far
2776649
Cumulative beneficiaries vaccinated 2nd dose so far
775388
COVID-19 Posi

In [19]:
import re
from datetime import datetime

def getDay(string) :
    s = ""
    for char in string:
        if char.isdigit():
            s += char

    return int(s)

def parseString(string):
    val = dict()
    try:
        #Date
        match=(re.search("/ ", string))
        i = int(match.end())
        match=(re.search("\)", string))
        j = int(match.start())
        print(string[i:j])
        val['day'] = getDay(string[i:string.find(str(datetime.today().year)) - 1])
        val['month'] = datetime.today().month
        val['year'] = datetime.today().year


        #+ve cases
        i = string.find("Positive Cases", string.find("Positive Cases")+1)
        match = (re.search("Tests Conducted", string))
        j = int(match.start())
        val['positive'] = string[i + len("Positive Cases") + 1 : j - 1]

        #Tests
        match=(re.search("Tests Conducted", string))
        i = int(match.end())
        match=(re.search("Positivity Rate", string))
        j = int(match.start())
        val['tests'] = string[i+1:j-1]

        #Recovered
        match=(re.search("%", string))
        i = int(match.end())
        match=(re.search("Deaths", string))
        j = int(match.start())
        val['recovered'] = string[i+1:j-1]

        #Deaths
        match=(re.search("Deaths", string))
        i = int(match.end())
        match=(re.search("COVID-19 Patient Management", string))
        j = int(match.start())
        val['deaths'] = string[i+1:j-1]

        #Tot Vac
        match=(re.search("Beneficiaries vaccinated in last 24 hours", string))
        i = int(match.end())
        match=(re.search("Beneficiaries vaccinated 1", string))
        j = int(match.start())
        val['vaccinated'] = string[i+1:j-1]

        #1st dose
        match=(re.search("dose in last 24 hours", string))
        i = int(match.end())
        match=(re.search("Beneficiaries vaccinated 2", string))
        j = int(match.start())
        val['first_dose'] = string[i+1:j-1]

        #2nd dose
        pat = "dose in last 24 hours"
        pos = string.find(pat)+1
        p1 = string.find(pat, pos+1)
        i = p1 + len(pat)
        match=(re.search("Cumulative beneficiaries vaccinated so far", string))
        j = int(match.start())
        val['second_dose'] = string[i+1:j-1]

        #Active Cases
        match=(re.search("Active Cases", string))
        i = int(match.end())
        match=(re.search("Total Number of Containment Zones as on date", string))
        j = int(match.start())
        val['active_cases'] = string[i+1:j-1]

        #Containment
        match=(re.search("Total Number of Containment Zones as on date: ", string))
        i = int(match.end())
        match=(re.search("Calls received", string))
        j = int(match.start())
        val['zones'] = string[i:j-1]
    except:
        print("/\/\/\ Error caught \/\/\/")
        return None
    print(val)
    return val

In [20]:
parseString(string)

May 6th 2021
{'day': 6, 'month': 5, 'year': 2021, 'positive': '19133', 'tests': '78780', 'recovered': '20028', 'deaths': '335', 'vaccinated': '68205', 'first_dose': '56857', 'second_dose': '11348', 'active_cases': '90629', 'zones': '49123'}


{'day': 6,
 'month': 5,
 'year': 2021,
 'positive': '19133',
 'tests': '78780',
 'recovered': '20028',
 'deaths': '335',
 'vaccinated': '68205',
 'first_dose': '56857',
 'second_dose': '11348',
 'active_cases': '90629',
 'zones': '49123'}

In [ ]:
import requests


